In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from scipy import stats
from sklearn.naive_bayes import GaussianNB

# Load the data
file_path = "dataset/agriculture_dataset.csv"
df = pd.read_csv(file_path)


In [3]:
# Define the function to handle missing values
def handle_missing_values(data, column, strategy='mean'):
    imputer = SimpleImputer(strategy=strategy)
    data[[column]] = imputer.fit_transform(data[[column]])
    return data

# Define the function for IQR outlier detection
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[f'IQR_outlier'] = np.where((data[column] < lower_bound) | (data[column] > upper_bound), -1, 1)
    return data

# Define the function for Z-Score outlier detection
def detect_outliers_zscore(data, column, threshold=3):
    data['ZScore'] = (data[column] - data[column].mean()) / data[column].std()
    data[f'ZScore_outlier'] = np.where(np.abs(data['ZScore']) > threshold, -1, 1)
    data.drop(columns=['ZScore'], inplace=True)
    return data

# Define the function for combined IQR and Z-Score outlier detection
def detect_outliers_iqr_zscore(data, column):
    data = detect_outliers_iqr(data, column)
    data = detect_outliers_zscore(data, column)
    data['IQR_ZScore_outlier'] = np.where((data['IQR_outlier'] == -1) & (data['ZScore_outlier'] == -1), -1, 1)
    return data

# Define the function for Isolation Forest outlier detection
def detect_outliers_isolation_forest(data, column):
    iso_forest = IsolationForest(contamination=0.05)
    labels = iso_forest.fit_predict(data[[column]])
    data[f'IF_outlier'] = np.where(labels == -1, -1, 1)
    return data

# Define the function for DBSCAN outlier detection
def detect_outliers_dbscan(data, column):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data[[column]])
    dbscan = DBSCAN(eps=0.3, min_samples=10)
    labels = dbscan.fit_predict(scaled_data)
    data[f'DBSCAN_outlier'] = np.where(labels == -1, -1, 1)
    return data

# Define the function for One-Class SVM outlier detection
def detect_outliers_one_class_svm(data, column):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data[[column]])
    svm = OneClassSVM(nu=0.05)
    labels = svm.fit_predict(scaled_data)
    data[f'OneClassSVM_outlier'] = np.where(labels == -1, -1, 1)
    return data

# Define the function for Local Outlier Factor (LOF) outlier detection
def detect_outliers_lof(data, column):
    lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
    labels = lof.fit_predict(data[[column]])
    data[f'LOF_outlier'] = np.where(labels == -1, -1, 1)
    return data


In [13]:
# Define the function for Gaussian Naive Bayes outlier detection
def detect_outliers_naive_bayesx(data, column, threshold=0.01):
    # Reshape the data to be 2D as expected by GaussianNB
    reshaped_data = data[[column]].values.reshape(-1, 1)
    
    # Fit the Gaussian Naive Bayes model
    gnb = GaussianNB()
    gnb.fit(reshaped_data, np.zeros(reshaped_data.shape[0]))
    
    # Get the probability of each point being an inlier
    inlier_prob = gnb.predict_proba(reshaped_data)[:, 1]
    
    # Label outliers as -1 if their inlier probability is below the threshold
    data[f'NaiveBayes_outlier'] = np.where(inlier_prob < threshold, -1, 1)
    
    return data

In [4]:
from scipy.stats import norm

# Define the function for Gaussian Naive Bayes outlier detection using the Gaussian PDF
def detect_outliers_naive_bayes(data, column, threshold=0.01):
    # Calculate the mean and standard deviation of the column
    mean = data[column].mean()
    std_dev = data[column].std()
    
    # Calculate the probability density for each point
    pdf_values = norm.pdf(data[column], mean, std_dev)
    
    # Label outliers as -1 if their probability density is below the threshold
    data[f'NaiveBayes_outlier'] = np.where(pdf_values < threshold, -1, 1)
    
    return data

In [5]:
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from pyod.models.hbos import HBOS
from pyod.models.cblof import CBLOF
from scipy.spatial.distance import mahalanobis

In [6]:
# HBOS (Histogram-based Outlier Score)
def detect_outliers_hbos(data, column):
    hbos = HBOS()
    hbos.fit(data[[column]])
    scores = hbos.decision_function(data[[column]])
    threshold = np.percentile(scores, 95)
    data[f'HBOS_outlier'] = np.where(scores > threshold, -1, 1)
    return data

# CBLOF (Cluster-Based Local Outlier Factor)
def detect_outliers_cblof(data, column):
    cblof = CBLOF()
    cblof.fit(data[[column]])
    labels = cblof.predict(data[[column]])
    data[f'CBLOF_outlier'] = np.where(labels == 1, 1, -1)  # CBLOF marks outliers as 1
    return data

# Mahalanobis Distance
def detect_outliers_mahalanobis(data, column):
    mean = np.mean(data[column])
    cov = np.cov(data[column].T)
    inv_covmat = np.linalg.inv(cov)
    data['Mahalanobis'] = data.apply(lambda row: mahalanobis(row[column], mean, inv_covmat), axis=1)
    threshold = np.percentile(data['Mahalanobis'], 95)
    data[f'Mahalanobis_outlier'] = np.where(data['Mahalanobis'] > threshold, -1, 1)
    data.drop(columns=['Mahalanobis'], inplace=True)
    return data


In [ ]:

# Apply the missing value handler
data = handle_missing_values(df, 'N2O', strategy='mean')


In [ ]:

# Apply each outlier detection method
data = detect_outliers_iqr(data, 'N2O')
data = detect_outliers_zscore(data, 'N2O')
data = detect_outliers_iqr_zscore(data, 'N2O')
data = detect_outliers_isolation_forest(data, 'N2O')
data = detect_outliers_dbscan(data, 'N2O')
data = detect_outliers_one_class_svm(data, 'N2O')
data = detect_outliers_lof(data, 'N2O')
data = detect_outliers_naive_bayes(data, 'N2O')

# Display the updated data with outlier column
data = detect_outliers_hbos(data, 'N2O')
data = detect_outliers_cblof(data, 'N2O')
data = detect_outliers_mahalanobis(data, 'N2O')


In [ ]:
data

In [ ]:
# Summarize the count of outliers and inliers for each method

def summarize_outliers(df, method_columns):
    summary = []
    for column in method_columns:
        inliers = df[column].value_counts().get(1, 0)
        outliers = df[column].value_counts().get(-1, 0) if df[column].min() == -1 else df[column].value_counts().get(0, 0)
        total = len(df[column])
        summary.append({
            'Method': column.replace('_outlier', ''),
            'Outliers': outliers,
            'Inliers': inliers,
            'Total': total
        })
    return pd.DataFrame(summary)

# List of outlier detection columns
outlier_columns = [col for col in df.columns if col.endswith('_outlier')]

# Generate the summary
summary_df = summarize_outliers(df, outlier_columns)

# import ace_tools as tools; tools.display_dataframe_to_user(name="Outlier Detection Summary", dataframe=summary_df)

summary_df


In [ ]:
# Define a function to summarize the count of inliers and outliers for each method
def summarize_outliers(data, methods):
    summary = []
    for method in methods:
        inliers = (data[f'{method}_outlier'] == 1).sum()
        outliers = (data[f'{method}_outlier'] == -1).sum()
        total = inliers + outliers
        summary.append({'Method': method, 'Inliers': inliers, 'Outliers': outliers, 'Total': total})
    return pd.DataFrame(summary)

# List of methods used for outlier detection
methods = ['IQR', 'ZScore', 'IQR_ZScore', 'IF', 'DBSCAN', 'OneClassSVM', 'LOF']

# Generate the summary
summary_df = summarize_outliers(data, methods)

# import ace_tools as tools; tools.display_dataframe_to_user(name="Outlier Detection Summary", dataframe=summary_df)

summary_df


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set up the figure layout
fig, axes = plt.subplots(3, 2, figsize=(15, 18))

# List of methods for outlier detection
methods = ['IQR', 'ZScore', 'IQR_ZScore', 'IF', 'DBSCAN', 'OneClassSVM', 'LOF']

# Define a function to plot outliers for each method
def plot_outliers(data, method, ax):
    sns.scatterplot(x=data.index, y=data['N2O'], hue=data[f'{method}_outlier'], palette={1: 'blue', -1: 'red'}, ax=ax)
    ax.set_title(f'{method} Outlier Detection')
    ax.set_xlabel('Index')
    ax.set_ylabel('N2O Levels')
    ax.legend(title='Outlier', loc='upper right')

# Create a plot for each method
for i, method in enumerate(methods[:6]):
    plot_outliers(data, method, axes[i // 2, i % 2])

# Adjust layout
plt.tight_layout()

# Display the plots
plt.show()


In [ ]:
# Redefine the plotting code to handle the correct number of subplots (3 rows x 2 columns for 6 methods)
# Removing one of the methods from the plotting to fit the layout or handle separately

# List of methods used for outlier detection (reduced to 6 for fitting 3x2 layout)
methods_to_plot = ['IQR', 'ZScore', 'IQR_ZScore', 'IF', 'DBSCAN', 'OneClassSVM']

# Create subplots with 3 rows and 2 columns
fig, axes = plt.subplots(3, 2, figsize=(15, 15))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each method
for i, method in enumerate(methods_to_plot):
    ax = axes[i]
    inliers = data[data[f'{method}_outlier'] == 1]['N2O']
    outliers = data[data[f'{method}_outlier'] == -1]['N2O']
    
    ax.hist(inliers, bins=30, alpha=0.5, label='Inliers', color='green')
    ax.hist(outliers, bins=30, alpha=0.5, label='Outliers', color='red')
    ax.set_title(f'{method} Outlier Detection')
    ax.set_xlabel('N2O')
    ax.set_ylabel('Count')
    ax.legend()

# Adjust layout to prevent overlapping titles and labels
plt.tight_layout()

# Display the plots
plt.show()


In [ ]:
# To ensure proper indexing, we'll adjust the loop to match the correct number of subplots and handle exceptions.

def plot_outlier_comparison_final(summary_df):
    fig, axes = plt.subplots(3, 2, figsize=(14, 18))
    fig.suptitle('Outlier Detection Methods Comparison', fontsize=16)
    axes = axes.flatten()
    methods = summary_df['Method']

    # Plot each method
    for i, method in enumerate(methods):
        if i < len(axes):  # Ensure we don't access out-of-bound indexes
            axes[i].bar(['Inliers', 'Outliers'], summary_df.loc[i, ['Inliers', 'Outliers']], color=['skyblue', 'salmon'])
            axes[i].set_title(f'{method} Method')
            axes[i].set_xlabel('Category')
            axes[i].set_ylabel('Count')
            axes[i].set_ylim(0, summary_df['Total'].max() + 100)

    # Hide any unused subplots
    for j in range(len(methods), len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Visualize the comparison with the final corrected version
plot_outlier_comparison_final(summary_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set up the layout for the subplots
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
fig.suptitle('Outlier Detection Methods Comparison', fontsize=16)

# Flatten axes for easier indexing
axes = axes.flatten()

# Method list and their corresponding titles for the plots
methods = ['IQR', 'ZScore', 'IQR_ZScore', 'IF', 'DBSCAN', 'OneClassSVM', 'LOF']
titles = ['IQR', 'Z-Score', 'IQR + Z-Score', 'Isolation Forest', 'DBSCAN', 'One-Class SVM', 'Local Outlier Factor']

# Generate boxplots for each method
for i, method in enumerate(methods):
    if i < len(axes):
        sns.boxplot(
            x=data[f'{method}_outlier'].astype(str),  # Convert labels to strings for the palette
            y=data['N2O'], 
            ax=axes[i], 
            palette={'1': 'lightblue', '-1': 'red'}
        )
        axes[i].set_title(titles[i])
        axes[i].set_xlabel('Outlier Label')
        axes[i].set_ylabel('N2O Levels')
        axes[i].set_xticks([0, 1])
        axes[i].set_xticklabels(['Outlier', 'Inlier'])

# Remove any empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# Adjust the layout for 4 rows and 2 columns to accommodate all methods
fig, axes = plt.subplots(4, 2, figsize=(15, 20))
plt.subplots_adjust(hspace=0.4, wspace=0.4)

# List of methods and corresponding names for visualization
methods_for_viz = ['IQR', 'ZScore', 'IQR_ZScore', 'IF', 'DBSCAN', 'OneClassSVM', 'LOF']
method_names = ['IQR', 'Z-Score', 'IQR & Z-Score', 'Isolation Forest', 'DBSCAN', 'One-Class SVM', 'LOF']

# Plot each method's outliers
for i, method in enumerate(methods_for_viz):
    row = i // 2
    col = i % 2
    sns.boxplot(x='N2O', y=f'{method}_outlier', data=data, ax=axes[row, col], orient='h')
    axes[row, col].set_title(f'{method_names[i]} Outlier Detection')
    axes[row, col].set_yticks([-1, 1])
    axes[row, col].set_yticklabels(['Outlier (-1)', 'Inlier (1)'])
    for tick in axes[row, col].get_yticklabels():
        tick.set_rotation(0)

# Hide the unused subplot if the number of methods is odd
if len(methods_for_viz) % 2 != 0:
    fig.delaxes(axes[3, 1])

plt.tight_layout()
plt.show()
